<a href="https://colab.research.google.com/github/Afanasyy/colab/blob/main/%D0%A1%D0%B0%D0%BC%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%961.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Самостоятельная работа №1

* **Срок сдачи**: 26.03.22
* **Куда отправлять**: logerk3@gmail.com
  * Сюда же можно писать по вопросам, связанным с работой
* **Что отправлять**: Jupyter Notebook (файл с расширением `.ipynb`), в котором находится решение данной работы


## Задачи

1. Найдите любые три статьи на Wikipedia.org
  * Примеры статей: https://ru.wikipedia.org/wiki/Luigi%E2%80%99s_Mansion_3, https://ru.wikipedia.org/wiki/Joy-Con
  * **NB**: Это лишь примеры! Вам необходимо найти свои 
3. Используя библиотеки `requests` и `BeautifulSoup`, скачайте данные три статьи и сохраните из них текст
  * \*Найдите все изображения и сохраните ссылки на них в файл `links.txt`
4. Используя библиотеку `re` (регулярные выражения), очистите сохранённые тексты от специальных символов (например, от знаков `'][@#%^` и т.д.) и от стоп-слов
5. Используя библиотеку `NLTK`, произведите токенизацию по словам и по предложениям
  * \*Найдите среднюю длину предложения
6. Используя библиотеку `pymorphy`, найдите леммы всех слов, сохраните полученные леммы в файл `lemmas.txt`
  * **NB**: Леммы должны идти через пробел: `лемма_1 лемма_2 ... лемма_N`
7. Найдите частоту встречаемости каждой леммы, то есть составьте частотный словарь
  * \*Выведите на экран лемму, которая встречается наибольшее количество раз
8. С помощью библиотеки `csv` сохраните частоты лем в файл `lemmas_frequencies.csv`
  * Файл должен иметь следующую структуру:

| Лемма   | Частота  |
|---------|----------|
| лемма_1 | 100      |
| лемма_2 | 200      |
| ...     | ...      |
| лемма_N | N        |


**NB**: Задания со знаком `*` необязательные, их выполнение принесёт дополнительные баллы

**NB №2**: Также приветствуются любые дополнительные шаги с Вашей стороны (использование дополнительныех библиотек, дополнительный анализ и т.д.)

In [ ]:
import requests, json, nltk, re, pymorphy2, string, csv
from bs4 import BeautifulSoup

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
def remove_stop_words(tokens: str, stop_words: list) -> list:
  clean_tokens = []
  for token in tokens:
    if token not in stop_words:
      clean_tokens.append(token)
  return clean_tokens

In [ ]:
url=["https://ru.wikipedia.org/wiki/%D0%92%D0%B8%D0%BA%D0%B8%D0%BF%D0%B5%D0%B4%D0%B8%D1%8F","https://ru.wikipedia.org/wiki/%D0%93%D0%B0%D0%BB%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%90%D0%BD%D0%B4%D1%80%D0%BE%D0%BC%D0%B5%D0%B4%D1%8B","https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D0%BA%D0%B5%D1%80"]

In [ ]:
res=[]
soup=[]
for i in url:
  res.append(requests.get(i))
  soup.append(BeautifulSoup(res[-1].text, 'html5lib'))

main_text = ''
for i in soup:
  for tag in i('p'):
   main_text += tag.text

with open("/content/links.txt", "w") as f:
  for i in soup:
    for j in i('img'):
      f.write(j['src']+'\n')

In [ ]:
sentences = nltk.tokenize.sent_tokenize(main_text, language='russian')
aveage_len=0
for i in sentences:
  regexp = r'\w+'
  tokens = re.findall(regexp, i)
  aveage_len+=len(tokens)
aveage_len/=len(sentences)
print(aveage_len)
res_txt = re.sub(r'[^а-яА-Я0-9\s]', '', main_text)


20.845378151260505


In [ ]:
russian_stop_words = nltk.corpus.stopwords.words('russian')

tokens = nltk.tokenize.word_tokenize(res_txt, language='russian')

clean_tokens = remove_stop_words(tokens, russian_stop_words)


In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
with open("/content/lemmas.txt", "w") as f:
  for i in clean_tokens:
    f.write(morph.parse(i)[0].normal_form+' ')

In [ ]:
def count_frequencies(tokens: list) -> dict:
  special_symbols = '<>«»-–+\\' + string.punctuation
  
  lemmas = [morph.parse(token)[0].normal_form for token in tokens 
            if token not in special_symbols and token not in russian_stop_words]
  
  return nltk.probability.FreqDist(lemmas)

In [ ]:
word_frequencies = count_frequencies(clean_tokens)
word_frequencies.most_common(1)

[('галактика', 183)]

In [ ]:
with open('/content/lemmas_frequencies.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["Лемма","Частота"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in word_frequencies:
      writer.writerow({fieldnames[0]:i,fieldnames[1]:word_frequencies[i]})